In [1]:

from common.read_spectral_common import (
    data_df_13_09_2022, 
    calculate_mutual_info_for_all, 
    generate_sample, create_X_train_Y_train, 
    mutual_info_regression, 
    get_max_bands, 
    get_bands_ix_from_mutual_info, 
    get_average_bands, 
    get_max_bands, 
    get_min_bands,
    predict_using_neutral_network, 
    predict_using_random_forest, 
    predict_using_decision_tree,
    get_full_path,
    load_sklearn_model_to_file_by_cloudpickle,
)
import matplotlib.pyplot as plt
import pickle
import cloudpickle
import common
from sklearn import tree
from sklearn.metrics import mean_squared_error
import numpy as np
from datetime import datetime
import os
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd

In [26]:

target_value = "N"
train_field = ["T", "J"]
function_get = get_max_bands

In [27]:
calculate_mutual_info_for_all(data_df_13_09_2022, target_value, train_field, function_get)

[(109, 0.266475167231325),
 (45, 0.196122954304089),
 (102, 0.15157344875394374),
 (21, 0.12335508977432275),
 (41, 0.11416500875691415),
 (93, 0.11012437169631761),
 (69, 0.10751346860020483),
 (24, 0.09428707544226489),
 (7, 0.09332524049104407),
 (63, 0.09158496679653805),
 (114, 0.09080279753805254),
 (96, 0.08926160096411273),
 (42, 0.08649622610630026),
 (106, 0.08215284543165557),
 (2, 0.07718416278583629),
 (12, 0.07123400334383456),
 (48, 0.06622563941400728),
 (37, 0.05957749549469726),
 (87, 0.058034064567277444),
 (53, 0.05525325130245262),
 (51, 0.05333506237671726),
 (23, 0.04664410309205991),
 (79, 0.042472031146980616),
 (98, 0.04115870796455834),
 (70, 0.04053388407511482),
 (35, 0.03791249713776956),
 (72, 0.03536779679315005),
 (57, 0.0348409780245591),
 (16, 0.03396583913210005),
 (18, 0.029266313625623486),
 (119, 0.027582159624412794),
 (4, 0.023798618055023812),
 (9, 0.023696899803736127),
 (28, 0.02034555660044024),
 (59, 0.019915797186019102),
 (97, 0.019414841

In [4]:
bands_ix = get_bands_ix_from_mutual_info(data_df_13_09_2022, -1, target_value, train_field, function_get)

In [5]:
len(bands_ix)

122

In [6]:
# bands_ix = filter(lambda data: data < 100, bands_ix)

In [7]:
bands_ix = list(bands_ix)

In [8]:
bands_ix

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121]

In [9]:
len(bands_ix)

122

In [10]:
date_now = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
NN_file_path = get_full_path(f"../../model_saved/NN_save/NN_object/{'_'.join(train_field)}_predict_{'_'.join(target_value)}_{date_now}_using_{function_get.__name__}.pkl")
RF_file_path = get_full_path(f"../../model_saved/RF_save/{'_'.join(train_field)}_predict_{'_'.join(target_value)}_{date_now}_using_{function_get.__name__}.pkl")
DT_file_path = get_full_path(f"../../model_saved/DT_save/{'_'.join(train_field)}_predict_{'_'.join(target_value)}_{date_now}_using_{function_get.__name__}.pkl")

In [11]:
sample = generate_sample(data_df_13_09_2022, bands_ix, target_value, train_field)
X_train, Y_train = create_X_train_Y_train(sample, bands_ix)
sample_target = generate_sample(data_df_13_09_2022, bands_ix, target_value, "BC")
X_target, Y_target = create_X_train_Y_train(sample_target, bands_ix)
super_param={"lr": 0.0001, "weight_decay": 1e-5, "n_epochs": 40000, "stop_value": 130}
re_run = "Y"
loss_NN, pred_NN, NN_model = predict_using_neutral_network(
    X_train, 
    Y_train, 
    X_target, 
    Y_target, 
    bands_ix,
    NN_file_path, 
    super_param, 
    re_run)
print(f"{loss_NN=}")
print(f"{pred_NN=}")
loss_RF, pred_RF, RF_model = predict_using_random_forest(X_train, Y_train, X_target, Y_target, bands_ix, super_param)
print(f"{loss_RF=}")
print(f"{pred_RF=}")
loss_DT, pred_DT, DT_model = predict_using_decision_tree(X_train, Y_train, X_target, Y_target, bands_ix, super_param)
print(f"{loss_DT=}")
print(f"{pred_DT=}")

Epoch: 0 | loss train: 5990.31640625
Epoch: 100 | loss train: 5735.78466796875
Epoch: 200 | loss train: 769.1438598632812
Epoch: 300 | loss train: 763.0464477539062
Epoch: 400 | loss train: 758.9034423828125
Epoch: 500 | loss train: 753.021728515625
Epoch: 600 | loss train: 735.8843383789062
Epoch: 700 | loss train: 703.10009765625
Epoch: 800 | loss train: 639.64599609375
Epoch: 900 | loss train: 579.9301147460938
Epoch: 1000 | loss train: 570.0950317382812
Epoch: 1100 | loss train: 561.340087890625
Epoch: 1200 | loss train: 551.3972778320312
Epoch: 1300 | loss train: 539.7272338867188
Epoch: 1400 | loss train: 526.0421752929688
Epoch: 1500 | loss train: 508.0052490234375
Epoch: 1600 | loss train: 487.75653076171875
Epoch: 1700 | loss train: 469.1561584472656
Epoch: 1800 | loss train: 453.9056701660156
Epoch: 1900 | loss train: 438.4959716796875
Epoch: 2000 | loss train: 426.2626037597656
Epoch: 2100 | loss train: 413.84735107421875
Epoch: 2200 | loss train: 405.3719787597656
Epoch: 23

In [23]:
cloudpickle.register_pickle_by_value(common)

with open(NN_file_path, "wb") as file:
    cloudpickle.dump(NN_model, file)
    print(f"Save NN object to: {NN_file_path}")
with open(RF_file_path, "wb") as file:
    cloudpickle.dump(RF_model, file)
    print(f"Save RF object to: {RF_file_path}")
with open(DT_file_path, "wb") as file:
    cloudpickle.dump(DT_model, file)
    print(f"Save DT object to: {DT_file_path}")

Save NN object to: D:\code\model_saved\NN_save\NN_object\T_predict_K_2024-03-07-22-56-00_using_get_max_bands.pkl
Save RF object to: D:\code\model_saved\RF_save\T_predict_K_2024-03-07-22-56-00_using_get_max_bands.pkl
Save DT object to: D:\code\model_saved\DT_save\T_predict_K_2024-03-07-22-56-00_using_get_max_bands.pkl


In [24]:
avarage_target_value = np.average(Y_target)
loss_NN = float(loss_NN)
df_report = pd.DataFrame(
    {
        "date_time": [datetime.now()],
        "train_field": [str(train_field)],
        "target_value": [str(target_value)],
        "bands_ix": [str(bands_ix)],
        "total_band": [len(bands_ix)],
        "function_get": [function_get.__name__],
        "loss_NN": [loss_NN],
        "% loss_NN": [loss_NN / avarage_target_value * 100],
        "loss_RF": [loss_RF],
        "% loss_RF": [loss_RF / avarage_target_value * 100],
        "loss_DT": [loss_DT],
        "% loss_DT": [loss_DT / avarage_target_value * 100],
        "average_target_value": [avarage_target_value],
        "File save NN": [NN_file_path],
        "File save RF": [RF_file_path],
        "File save DT": [DT_file_path],
    }
)

In [25]:
df_report

,date_time,train_field,target_value,bands_ix,total_band,function_get,loss_NN,% loss_NN,loss_RF,% loss_RF,loss_DT,% loss_DT,average_target_value,File save NN,File save RF,File save DT
0,2024-03-07 23:02:15.481179,T,K,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",122,get_max_bands,4755.72998,37.707614,3506.90181,27.805805,3099.819108,24.578095,12612.121094,D:\code\model_saved\NN_save\NN_object\T_predic...,D:\code\model_saved\RF_save\T_predict_K_2024-0...,D:\code\model_saved\DT_save\T_predict_K_2024-0...


In [26]:
def append_df_to_excel(df: pd.DataFrame, file_path, sheet_name="Sheet1"):
    if not os.path.exists(file_path):
        df.to_excel(
            file_path,
            sheet_name=sheet_name,
            index=False,
            header=True
        )
        return
    
    wb = load_workbook(file_path)
    ws = wb[sheet_name]
    for r in dataframe_to_rows(df, index=False, header=False):
        ws.append(r)
    wb.save(file_path)

In [27]:
try:
    report_file_path = get_full_path("../../report/agriculture_report.xlsx")
    sheet_name = "Sheet1"

    append_df_to_excel(df_report, report_file_path, sheet_name)
    print(f"SUCCESS: DATA IS WRITED TO FILE {report_file_path}")
except PermissionError as err:
    print(f"ERROR: YOU ARE OPENNING FILE {report_file_path}")

SUCCESS: DATA IS WRITED TO FILE D:\code\report\agriculture_report.xlsx
